In [1]:
spark

In [3]:
infile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/0.with_corrected_dates'
df0 = spark.read.format('csv').option('header','true').load(infile)
df0.printSchema()

root
 |-- msno: string (nullable = true)
 |-- pay_method: string (nullable = true)
 |-- plan_days: string (nullable = true)
 |-- list_price: string (nullable = true)
 |-- actual_paid: string (nullable = true)
 |-- auto_renew: string (nullable = true)
 |-- is_cancel: string (nullable = true)
 |-- trans_date: string (nullable = true)
 |-- expire_date: string (nullable = true)
 |-- corr_exp_date: string (nullable = true)
 |-- corr_start_date: string (nullable = true)



In [4]:
df0.show(5)

+--------------------+----------+---------+----------+-----------+----------+---------+----------+-----------+-------------+---------------+
|                msno|pay_method|plan_days|list_price|actual_paid|auto_renew|is_cancel|trans_date|expire_date|corr_exp_date|corr_start_date|
+--------------------+----------+---------+----------+-----------+----------+---------+----------+-----------+-------------+---------------+
|0jqpsCaqfNLSrrqeB...|        38|       30|       149|        149|     false|    false|2015-11-09| 2015-12-09|   2015-12-09|     2015-11-10|
|kT+9EbT/Lf11zKmTY...|        41|       30|        99|         99|      true|    false|2016-06-22| 2016-07-22|   2016-07-22|     2016-06-23|
|BBeTn8ptj/mnjIHF7...|        36|       30|       180|        180|     false|    false|2016-05-21| 2016-06-20|   2016-06-20|     2016-05-22|
|dcPFeF1EN5i0Ylwfa...|        41|       30|        99|         99|      true|    false|2016-10-21| 2016-11-22|   2016-11-22|     2016-10-24|
|An7+T0ShT7SU

In [12]:
# filter corr_exp_date != null
from pyspark.sql.functions import col
df1 = df0.where(~col('corr_exp_date').isNull())
df1.show(5)

+--------------------+----------+---------+----------+-----------+----------+---------+----------+-----------+-------------+---------------+
|                msno|pay_method|plan_days|list_price|actual_paid|auto_renew|is_cancel|trans_date|expire_date|corr_exp_date|corr_start_date|
+--------------------+----------+---------+----------+-----------+----------+---------+----------+-----------+-------------+---------------+
|0jqpsCaqfNLSrrqeB...|        38|       30|       149|        149|     false|    false|2015-11-09| 2015-12-09|   2015-12-09|     2015-11-10|
|kT+9EbT/Lf11zKmTY...|        41|       30|        99|         99|      true|    false|2016-06-22| 2016-07-22|   2016-07-22|     2016-06-23|
|BBeTn8ptj/mnjIHF7...|        36|       30|       180|        180|     false|    false|2016-05-21| 2016-06-20|   2016-06-20|     2016-05-22|
|dcPFeF1EN5i0Ylwfa...|        41|       30|        99|         99|      true|    false|2016-10-21| 2016-11-22|   2016-11-22|     2016-10-24|
|An7+T0ShT7SU

In [44]:
df1.groupBy('msno').count().count()

1943654

In [14]:
df2 = df1.select('msno', 'is_cancel', 'trans_date')
df2.show(10)

+--------------------+---------+----------+
|                msno|is_cancel|trans_date|
+--------------------+---------+----------+
|0jqpsCaqfNLSrrqeB...|    false|2015-11-09|
|kT+9EbT/Lf11zKmTY...|    false|2016-06-22|
|BBeTn8ptj/mnjIHF7...|    false|2016-05-21|
|dcPFeF1EN5i0Ylwfa...|    false|2016-10-21|
|An7+T0ShT7SUx3yGI...|    false|2017-01-20|
|FGE0LQpNE5MXUJr4t...|    false|2016-02-19|
|syKvzihN0s1ry+RF8...|    false|2017-01-31|
|mPJ0OG5RIAWhnQ1aw...|    false|2016-09-30|
|tBjvY3e48CkVwjZRh...|    false|2017-01-13|
|Q3KfGQbfE8Wc1qCgO...|    false|2016-09-14|
+--------------------+---------+----------+
only showing top 10 rows



In [17]:
from pyspark.sql.functions import sum, when, col, count
df3 = df2.groupBy('msno', 'trans_date') \
         .agg(count('*').alias('count'),
              sum(when(col('is_cancel') == 'true', 1).otherwise(0)).alias('sum_is_cancel')
             )
df3.show(10)

+--------------------+----------+-----+-------------+
|                msno|trans_date|count|sum_is_cancel|
+--------------------+----------+-----+-------------+
|S5wPzT1VKk6q7Ug+v...|2015-11-20|    1|            0|
|Z1Z2OCoa3V8igCIe+...|2015-11-27|    1|            0|
|ICPWkZSBvrliGU7VE...|2016-03-31|    1|            0|
|44944J4eX5pJe5wBG...|2016-05-20|    1|            0|
|7h0hrPl8JR/sdbt21...|2015-12-06|    1|            0|
|jO+2AfJPTC6iWBelw...|2016-12-20|    1|            0|
|iJKyIuKGCyQsESQri...|2017-02-09|    1|            0|
|sfFxkInJJhz2uyJyK...|2016-10-01|    1|            0|
|tlwLnlHV1MznR6GxK...|2016-07-31|    1|            0|
|4dS2GxFTdsdgGbSGs...|2015-12-25|    1|            0|
+--------------------+----------+-----+-------------+
only showing top 10 rows



In [18]:
df4 = df3.where('count > 1').persist()
df4.count()

171452

In [19]:
df4.show(10)

+--------------------+----------+-----+-------------+
|                msno|trans_date|count|sum_is_cancel|
+--------------------+----------+-----+-------------+
|KtSvNZ0IXxwHfpbig...|2016-02-25|    2|            1|
|hRvC5cdmIBPeKG3rX...|2015-11-27|    2|            0|
|BbSFtRPSDVfy6X4yW...|2016-08-13|    2|            0|
|myN3qDk4+GBsWyMJk...|2016-03-20|    2|            0|
|LpSDsDdM7B7FLeSA0...|2015-11-27|    2|            1|
|wt8ufqk+SB6tJvs55...|2015-11-22|    2|            1|
|UlpOGLJHzPKRJfQlE...|2016-08-27|    3|            0|
|/BTl22m74NcOwaiAj...|2016-11-26|    2|            1|
|/MhTxrQTw1/6Oed/K...|2016-02-06|    2|            1|
|/QWUxn+BdzBjvhLtQ...|2016-07-17|    2|            0|
+--------------------+----------+-----+-------------+
only showing top 10 rows



In [47]:
# (member, date) pairs with at least one non-cancel and at least one cancel
df5 = df4.where('sum_is_cancel > 0').where('sum_is_cancel < count') \
  .withColumnRenamed('count', 'count_records')
df5.count()

129846

In [48]:
df5.show(10)

+--------------------+----------+-------------+-------------+
|                msno|trans_date|count_records|sum_is_cancel|
+--------------------+----------+-------------+-------------+
|KtSvNZ0IXxwHfpbig...|2016-02-25|            2|            1|
|LpSDsDdM7B7FLeSA0...|2015-11-27|            2|            1|
|wt8ufqk+SB6tJvs55...|2015-11-22|            2|            1|
|/BTl22m74NcOwaiAj...|2016-11-26|            2|            1|
|/MhTxrQTw1/6Oed/K...|2016-02-06|            2|            1|
|08lliUyrewyx+wz+v...|2015-11-03|            2|            1|
|11hzJPnhs6HAGUDez...|2015-11-12|            2|            1|
|1kIg0DIU6NHG7JX6L...|2015-12-29|            2|            1|
|2mq0bS+n+qxwquDWV...|2016-10-03|            2|            1|
|2s0AD9TskaN3+KVup...|2015-11-16|            2|            1|
+--------------------+----------+-------------+-------------+
only showing top 10 rows



In [49]:
df5.groupBy('count_records').count().show()

+-------------+------+
|count_records| count|
+-------------+------+
|            7|     9|
|            6|    20|
|           27|     1|
|           17|     1|
|            5|    75|
|           10|     1|
|            3|  9489|
|            8|     1|
|           11|     3|
|            2|119059|
|            4|  1184|
|           13|     1|
|           18|     1|
|           15|     1|
+-------------+------+



In [52]:
df6 = df5.where('count_records >= 3')
df7 = df6.groupBy('msno').count() \
      .drop('count')
df7.show(3)

# members with one date on which there are >= 3 records, of which at least one is non-cancel and at least one is cancel

+--------------------+
|                msno|
+--------------------+
|2NboYdCowDCzxTrRe...|
|rR8r2s9Wc92OScoaw...|
|2Wh84JFOH9raMfNZk...|
+--------------------+
only showing top 3 rows



In [54]:
df7.count()

10775

In [55]:
# output problematic members
outfile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/problematic_members'
df7.coalesce(1).write.format('csv').option('header','true').save(outfile)